In [9]:
from ase import Atoms
from ase.data import atomic_masses

def calculate_density(atoms: Atoms) -> float:
    """
    Calculate the density of an atomic structure represented by an ASE Atoms object.

    Parameters:
    atoms (Atoms): ASE Atoms object representing the atomic structure.

    Returns:
    float: Density of the atomic structure in g/cm³.
    """
    # Calculate the total mass of the atoms in the unit cell
    mass = sum(atomic_masses[atom.number] for atom in atoms)

    # Get the volume of the unit cell in cubic Ångströms
    volume = atoms.get_volume()

    # Convert mass to grams (since atomic masses are in atomic mass units, which is equivalent to g/mol)
    mass_in_grams = mass * 1.66054e-24  # 1 amu = 1.66054e-24 g

    # Convert volume to cm³ (since 1 Å³ = 1e-24 cm³)
    volume_in_cm3 = volume * 1e-24

    # Calculate density in g/cm³
    density = mass_in_grams / volume_in_cm3

    return density

In [11]:
from ase import Atoms
import ase.io

# More accurate coordinates for H2O
water = Atoms('H2O',
              positions=[(0.0, 0.0, 0.0),       # Oxygen
                         (0.9572, 0.0, 0.0),    # Hydrogen 1
                         (-0.2399872, 0.9266272, 0.0)])  # Hydrogen 2
calculate_density(water)

ValueError: You have 0 lattice vectors: volume not defined

In [13]:
from ase import Atoms
from ase.build import molecule, make_supercell
import ase.io

# Create a single water molecule
water = molecule('H2O')

# Define the unit cell size (arbitrary example)
a = 5.0
b = 5.0
c = 5.0

# Place the single water molecule in a unit cell
water.set_cell([a, b, c], scale_atoms=True)
water.center()

# Define the supercell matrix to create a 3x3x3 simulation box
P = [[3, 0, 0],
     [0, 3, 0],
     [0, 0, 3]]

# Create the supercell
water_box = make_supercell(water, P)
calculate_density(water_box)

0.23931702480000003

In [15]:
water_box.get_number_of_atoms()
from ase.build import bulk
copper = bulk('Cu', 'fcc', a=3.6)
calculate_density(copper)

/tmp/ipykernel_281924/1907936983.py:1: FutureWarning: Please use len(self) or, if your atoms are distributed, self.get_global_number_of_atoms.
  water_box.get_number_of_atoms()


9.046697088477368

In [7]:
from mp_api.client import MPRester
API_KEY = "ETq6U0uXhv64qL43JJOMS7udnzhwUSjI"

with MPRester(api_key=API_KEY) as mpr:
    structure = mpr.get_structure_by_material_id("mp-22905")
    structure.to(fmt="poscar", filename="POSCAR_LiCl")
# Replace with your Materials Project API key

# # Initialize the MPRester with your API key
# mpr = MPRester(API_KEY)

# # Query the LiCl structure
# structure = mpr.get_structure_by_material_id("mp-22905")  # Replace with the correct material_id for LiCl

# # Write the structure to a POSCAR file
# structure.to(fmt="poscar", filename="POSCAR_LiCl")

Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 5777.28it/s]


In [1]:
%pip install --upgrade MDAnalysis



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from pymatgen.core import Structure
from pymatgen.io.vasp import Poscar
import random

# Load the LiCl structure from a POSCAR file or create it
structure = Structure.from_file("POSCAR_LiCl")

# Define the doping concentration
doping_concentration = 0.02  # 2%
num_atoms = len(structure)
num_fe = int(num_atoms * doping_concentration / 2)

# Randomly replace Li with Fe and Cl with Fe
li_indices = [i for i, site in enumerate(structure) if site.species_string == "Li"]
cl_indices = [i for i, site in enumerate(structure) if site.species_string == "Cl"]

fe_li_indices = random.sample(li_indices, num_fe)
fe_cl_indices = random.sample(cl_indices, num_fe)

for i in fe_li_indices:
    structure.replace(i, "Fe")

for i in fe_cl_indices:
    structure.replace(i, "Fe")

# Save the doped structure to a POSCAR file
poscar = Poscar(structure)
poscar.write_file("POSCAR_doped")


In [9]:
from ase import Atoms
from ase.build import bulk
import random

# Create a bulk LiCl structure
structure = bulk('LiCl', 'rocksalt', a=5.64, cubic=True)

# Define the doping concentration
doping_concentration = 0.02  # 2%
num_atoms = len(structure)
num_fe = int(num_atoms * doping_concentration / 2)

# Randomly replace Li with Fe and Cl with Fe
li_indices = [i for i, atom in enumerate(structure) if atom.symbol == 'Li']
cl_indices = [i for i, atom in enumerate(structure) if atom.symbol == 'Cl']

fe_li_indices = random.sample(li_indices, num_fe)
fe_cl_indices = random.sample(cl_indices, num_fe)

for i in fe_li_indices:
    structure[i].symbol = 'Fe'

for i in fe_cl_indices:
    structure[i].symbol = 'Fe'

# Write the structure to a POSCAR file
structure.write('POSCAR_doped', format='vasp')


In [3]:
from ase import Atoms
from ase.build import bulk, make_supercell
import random

# Create a bulk LiCl structure
unit_cell = bulk('LiCl', 'rocksalt', a=5.64, cubic=True)

# Define the doping concentration
doping_concentration = 0.02  # 2%

# Calculate the number of formula units needed to get a reasonable supercell size
# We want the number of atoms to be large enough to accommodate the doping concentration
num_formula_units = int(1 / doping_concentration)  # e.g., 1/0.02 = 50

# Calculate the required supercell dimensions
# LiCl has 2 atoms per unit cell (1 Li and 1 Cl)
# We aim for a supercell that has at least num_formula_units atoms
# We'll take the cubic root to get the dimensions (approximation)
num_atoms_per_unit_cell = len(unit_cell)
print(f"Number of atoms per unit cell: {num_atoms_per_unit_cell}") 

supercell_dimension = int((num_formula_units * num_atoms_per_unit_cell) ** (1/3)) + 1

print(f"Supercell dimension: {supercell_dimension}") 
# Create the supercell
supercell = make_supercell(unit_cell, [[supercell_dimension, 0, 0], 
                                       [0, supercell_dimension, 0], 
                                       [0, 0, supercell_dimension]])

# Update the number of atoms in the supercell
num_atoms = len(supercell)
print(f"Number of atoms: {supercell_dimension}") 

num_fe = int(num_atoms * doping_concentration / 2)

# Randomly replace Li with Fe and Cl with Fe
li_indices = [i for i, atom in enumerate(supercell) if atom.symbol == 'Li']
cl_indices = [i for i, atom in enumerate(supercell) if atom.symbol == 'Cl']

fe_li_indices = random.sample(li_indices, num_fe)
fe_cl_indices = random.sample(cl_indices, num_fe)

for i in fe_li_indices:
    supercell[i].symbol = 'Fe'

for i in fe_cl_indices:
    supercell[i].symbol = 'Fe'

# Write the structure to a POSCAR file
#supercell.write('POSCAR_doped', format='vasp')


In [6]:
from ase.data import atomic_masses

print(supercell.get_global_number_of_atoms())
# Calculate the total mass of the atoms in the unit cell
mass = sum(atomic_masses[atom.number] for atom in supercell)

# Get the volume of the unit cell in cubic Ångströms
volume = supercell.get_volume()

# Convert mass to grams (since atomic masses are in atomic mass units, which is equivalent to g/mol)
mass_in_grams = mass * 1.66054e-24  # 1 amu = 1.66054e-24 g

# Convert volume to cm³ (since 1 Å³ = 1e-24 cm³)
volume_in_cm3 = volume * 1e-24

# Calculate density in g/cm³
density = mass_in_grams / volume_in_cm3

print(f'Density: {density} g/cm³')

4096
Density: 1.619517722557749 g/cm³


In [1]:
import MDAnalysis as mda
import mdapackmol

# load individual molecule files
water = mda.Universe('water.pdb')
urea = mda.Universe('urea.pdb')

# call Packmol with MDAnalysis objects as arguments
# the 'instructions' allow for any valid Packmol commands
system = mdapackmol.packmol(
    [mdapackmol.PackmolStructure(
        water, number=1000,
        instructions=['inside box 0. 0. 0. 40. 40. 40.']),
     mdapackmol.PackmolStructure(
        urea, number=400,
        instructions=['inside box 0. 0. 0. 40. 40. 40.'])]
)

# the returned system is a MDAnalysis Universe
# with all topology information from building blocks retained
# which can then be saved into any format
# eg to Lammps data file:
system.atoms.write('topology.data')

/home/keceli/venv/chemmat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/keceli/venv/chemmat/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/home/keceli/venv/chemmat/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/keceli/venv/chemmat/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warnings.warn("Found no i

ValueError: LAMMPS.DATAWriter: atom types must be convertible to integers